In [6]:
import numpy as np
import scipy.stats as sts


class MonteCarloOptionPricing:
    def __init__(self, r, S0, K, T, mue, sigma, div_yield=0.0, simulation_rounds=20000, no_of_slices=4,
                 fix_random_seed=False):
        """
        An important reminder, here the models rely on the assumption of constant interest rate and volatility.
        S0: current price of the underlying asset (e.g. stock)
        K: exercise price
        T: time to maturity, in years, can be float
        r: interest rate, here we assume constant interest rate model
        sigma: volatility (in standard deviation) of the asset annual returns
        div_yield: annual dividend yield
        simulation_rounds: in general, monte carlo option pricing requires many simulations
        no_of_slices: between time 0 and time T, the number of slices, e.g. 252 if trading days are required
        fix_random_seed: boolean, True or False
        """
        assert sigma >= 0, 'volatility cannot be less than zero'
        assert S0 >= 0, 'initial stock price cannot be less than zero'
        assert T >= 0, 'time to maturity cannot be less than zero'
        assert div_yield >= 0, 'dividend yield cannot be less than zero'
        assert no_of_slices >= 0, 'no of slices must be greater than zero'
        assert simulation_rounds >= 0, 'simulation rounds must be greater than zero'

        self.S0 = float(S0)
        self.K = float(K)
        self.T = float(T)
        self.mue = float(mue)
        self.div_yield = float(div_yield)

        self.no_of_slices = int(no_of_slices)
        self.simulation_rounds = int(simulation_rounds)

        self.r = np.full((self.simulation_rounds, self.no_of_slices), r / (self.T * self.no_of_slices))
        self.sigma = np.full((self.simulation_rounds, self.no_of_slices), sigma)
        
        
        self.h = self.T / self.no_of_slices

        self.terminal_prices = []

        if fix_random_seed:
            np.random.seed(15000)

    def stock_price_simulation(self):
        """
        :return:
        """
        self.exp_mean = (self.mue - self.div_yield - (self.sigma ** 2.0) * 0.5) * self.h
        self.exp_diffusion = self.sigma * np.sqrt(self.h)

        self.z_t = np.random.standard_normal((self.simulation_rounds, self.no_of_slices))
        self.price_array = np.zeros((self.simulation_rounds, self.no_of_slices))
        self.price_array[:, 0] = self.S0

        for i in range(1, self.no_of_slices):
            self.price_array[:, i] = self.price_array[:, i - 1] * np.exp(
                self.exp_mean[:, i] + self.exp_diffusion[:, i] * self.z_t[:, i]
            )

        self.terminal_prices = self.price_array[:, -1]
        self.stock_price_expectation = np.mean(self.terminal_prices)
        self.stock_price_standard_error = np.std(self.terminal_prices) / np.sqrt(len(self.terminal_prices))

        print('-' * 64)
        print(
            " Number of simulations %4.1i \n S0 %4.1f \n T %2.1f \n Maximum Stock price %4.2f \n"
            " Minimum Stock price %4.2f \n Average stock price %4.3f \n Standard Error %4.5f " % (
                self.simulation_rounds, self.S0, self.T, np.max(self.terminal_prices),
                np.min(self.terminal_prices), self.stock_price_expectation, self.stock_price_standard_error
            )
        )
        print('-' * 64)

        return self.stock_price_expectation, self.stock_price_standard_error

    def american_option_monte_carlo(self, poly_degree=2, option_type='call'):
        """
        American option
          poly_degree: x^n, default = 2
          option_type: x^n, default = 2
        :return:
        """
        assert option_type == 'call' or option_type == 'put', 'option_type must be either call or put'
        assert len(self.terminal_prices) != 0, 'Please simulate the stock price first'

        self.dis_factor = np.exp(- self.r * self.h)  # discount factor per time time interval

        if option_type == 'call':
            self.intrinsic_val = np.maximum((self.price_array - self.K), 0.0)
        elif option_type == 'put':
            self.intrinsic_val = np.maximum((self.K - self.price_array), 0.0)

        self.value_matrix = np.zeros_like(self.intrinsic_val)  # sample shape
        self.value_matrix[:, -1] = self.intrinsic_val[:, -1]  # last day american option value = intrinsic value

        # Longstaff and Schwartz
        for t in range(self.no_of_slices - 2, 0, -1):  # fill out the value table from backwards
            self.rg = np.polyfit(x=self.price_array[:, t], y=self.value_matrix[:, t + 1] * self.dis_factor[:, t + 1],
                                 deg=poly_degree)  # regression fitting
            self.hold_val = np.polyval(p=self.rg, x=self.price_array[:, t])  # regression estimated value

            # determine hold or exercise
            self.value_matrix[:, t] = np.where(self.intrinsic_val[:, t] > self.hold_val, self.intrinsic_val[:, t],
                                               self.value_matrix[:, t + 1] * self.dis_factor[:, t + 1])

        self.american_call_val = np.average(self.value_matrix[:, 1] * self.dis_factor[:, 1])
        self.am_std_error = np.std(self.value_matrix[:, 1] * self.dis_factor[:, 1]) / np.sqrt(self.simulation_rounds)

        print('-' * 64)
        print(
            " American %s \n polynomial degree = %i \n S0 %4.1f \n T %2.1f \n "
            "Call Option Value %4.3f \n Standard Error %4.5f " % (
                option_type, poly_degree, self.S0, self.T, self.american_call_val, self.am_std_error
            )
        )
        print('-' * 64)

        return self.american_call_val, self.am_std_error


In [7]:
# initialize parameters
T   = 1.0  # interval
r   = 0.05
K   = 100.0
S0 = 100.0  # spot price = 35
mue = 0.05  # expected return of the asset, under risk neutral assumption, mue = r
r = 0.05  # risk free rate
sigma = 0.2  # volatility
div_yield = 0.0  # dividend yield = 1%
no_of_slice = 252  # quarterly adjusted

# optional parameter
simulation_rounds = 20000

MT = MonteCarloOptionPricing(r, S0, K, T, mue, sigma, div_yield, simulation_rounds=simulation_rounds,
                             no_of_slices=no_of_slice, fix_random_seed=True)

MT.stock_price_simulation()

MT.american_option_monte_carlo(poly_degree=2, option_type='put')

----------------------------------------------------------------
 Number of simulations 20000 
 S0 100.0 
 T 1.0 
 Maximum Stock price 218.25 
 Minimum Stock price 45.31 
 Average stock price 104.944 
 Standard Error 0.14882 
----------------------------------------------------------------
----------------------------------------------------------------
 American put 
 polynomial degree = 2 
 S0 100.0 
 T 1.0 
 Call Option Value 5.729 
 Standard Error 0.04904 
----------------------------------------------------------------


(5.728780427094885, 0.049040766770697033)